In [1]:
import numpy as np
import torch
from mcspace.model import MCSPACE
from mcspace.trainer import train_model
from mcspace.data_utils import get_data, get_mouse_diet_perturbations_dataset
from mcspace.utils import get_device, pickle_load, pickle_save,\
    estimate_process_variance, MODEL_FILE, DATA_FILE, \
    get_mcspace_cooccur_prob, get_sig_perturbation_diff_subset, \
    get_posterior_summary_data, get_pert_otu_sub, get_min_loss_path, apply_taxonomy_threshold, \
    get_summary_results
from pathlib import Path
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
import mcspace.visualization as vis
from mcspace.dataset import DataSet
import pandas as pd

import ete3
from Bio import SeqIO, Phylo

import mcspace.vis_tools as vis
import matplotlib.colors as mcolors

In [2]:
# change font
mpl.rcParams['font.sans-serif'] = "Arial"
mpl.rcParams['font.family'] = "sans-serif"

# Paths

In [3]:
rootpath = Path("../../")
basepath = Path("./")

runpath = rootpath / "paper" / "mouse_runs" / "runs"

treepath = rootpath / "data" / "mouse_experiments" / "output" # *** rename this
treefile = "newick_tree_query_reads.nhx"

outpath = basepath / "figs_mouse_results"
outpath.mkdir(exist_ok=True, parents=True)

# Load model

In [4]:
respath = get_min_loss_path(runpath, np.arange(10))

model = torch.load(respath / MODEL_FILE)
data = pickle_load(respath / DATA_FILE)

taxonomy = pickle_load(respath / "taxonomy.pkl")

times = list(data['count_data'].keys())
subjects = list(data['count_data'][10].keys())
num_otus = data['count_data'][times[0]][subjects[0]].shape[1]
num_times = len(times)
num_subjects = len(subjects)

taxonomy = apply_taxonomy_threshold(taxonomy)

# update taxonomy names (do before hand -> update main taxonmy file)
name_updates = {'Otu10': {'Species': 'Faecalibaculum rodentium'},
'Otu17': {'Genus': 'Roseburia'},
'Otu6': {'Species': 'Eubacterium coprostanoligenes'},
'Otu20': {'Species': 'Muribaculum gordoncarteri'},
'Otu15': {'Genus': 'Eisenbergiella'},
'Otu43': {'Family': 'Lachnospiraceae'}}
taxonomy['Species'] = 'na'

for oidx in name_updates.keys():
    replace = name_updates[oidx]
    key = list(replace.keys())[0]
    newname = replace[key]
    taxonomy.loc[oidx,key] = newname
    
print("getting posterior summary...")
# get summary results
thetadf, betadf, pertsdf = get_posterior_summary_data(model, data, taxonomy, times, subjects)

0
1
2
3
4
5
6
7
8
9
4


# Render figure

In [5]:
betadf

,Time,Subject,Assemblage,Value,log10Value
0,10,JX07,A1,0.000403,-3.394213
1,10,JX07,A2,0.000467,-3.330563
2,10,JX07,A3,0.000324,-3.489476
3,10,JX07,A4,0.001320,-2.879548
4,10,JX07,A5,0.000465,-3.332299
...,...,...,...,...,...
1591,76,JX10,A72,0.029796,-1.525841
1592,76,JX10,A73,0.059429,-1.226000
1593,76,JX10,A74,0.006139,-2.211879
1594,76,JX10,A75,0.000310,-3.509136


In [6]:
pertsdf

,P1,P2,P3
A1,2.971620,2.640027,625.652710
A2,3.923855,3.964989,2159.749512
A3,2.907442,3.083910,3.269230
A4,2.746188,1.826614,2.431775
A5,6.023514,2.898376,3.637626
...,...,...,...
A72,3.238622,3.360882,3.568530
A73,6.738834,2.843930,5.138291
A74,3.501268,2.905026,3.239553
A75,2.782486,2.695037,12.365883
